__How to train a language model__	Notebook to Highlight all the steps to effectively train Transformer model on custom data
https://github.com/huggingface/transformers/tree/master/notebooks

https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb

__Language Modeling__
https://github.com/huggingface/transformers/tree/master/examples/language-modeling
https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_language_modeling.py

In [24]:
from pathlib import Path
import torch

from torch.utils.data import Dataset, DataLoader
from tokenizers import CharBPETokenizer
from tokenizers.processors import BertProcessing
from tokenizers.normalizers import BertNormalizer

from transformers import RobertaTokenizerFast, RobertaTokenizer

# Making Electra Model

However, currently pretraining Electra is still inside PR stage  
- Issue : When will ELECTRA pretraining from scratch will be available? #3878 https://github.com/huggingface/transformers/issues/3878  
- Issue : BERT and other models pretraining from scratch example #4425 https://github.com/huggingface/transformers/issues/4425
- PR : Electra training from scratch #4656 https://github.com/huggingface/transformers/pull/4656

Combined model
- Combines ElectraForMaskedLM and ElectraForPreTraining with embedding sharing + custom masking/replaced token detection

Commit before Merge: https://github.com/huggingface/transformers/pull/4656/commits/30b2dbbba6918ac6540b6a1758b7ee19f0ac969c#diff-8a95e2bfb7da25648b5d12ffa69fd7a3

In [5]:
from transformers import ElectraModel, ElectraConfig

# Initializing a ELECTRA electra-base-uncased style configuration
configuration = ElectraConfig()
configuration.vocab_size = 20000

# Initializing a model from the electra-base-uncased style configuration
model = ElectraModel(configuration)

# Accessing the model configuration
configuration = model.config
configuration

ElectraConfig {
  "attention_probs_dropout_prob": 0.1,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 20000
}

In [6]:
model

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(20000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=256, out_features=256, bias=True)
            (key): Linear(in_features=256, out_features=256, bias=True)
            (value): Linear(in_features=256, out_features=256, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=256, out_features=256, bias=True)
            (LayerNorm): LayerNorm((256,), eps=1e-12, eleme

In [8]:
model.num_parameters()
# => 12 million parameters

12136192

In [35]:
model.get_input_embeddings()

Embedding(20000, 128, padding_idx=0)

# Trying out Roberta per Notebook 

From __HuggingFace Notebooks__ https://huggingface.co/transformers/notebooks.html: 

How to train a language model	Highlight all the steps to effectively train Transformer model on custom data
- Colab (ipynb) version : https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb
- MD version: https://github.com/huggingface/blog/blob/master/how-to-train.md

Pretrain Longformer	How to build a "long" version of existing pretrained models	Iz Beltagy  
https://github.com/allenai/longformer/blob/master/scripts/convert_model_to_long.ipynb

In [2]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

configuration = RobertaConfig(
#     vocab_size=20000,
    max_position_embeddings=514, # 512 + 2 more special tokens
    num_attention_heads=12,
    num_hidden_layers=12,
    type_vocab_size=1,
)
# configuration.vocab_size = 20000

model = RobertaForMaskedLM(config=configuration)

# Accessing the model configuration
configuration = model.config
configuration

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 30522
}

In [3]:
model.num_parameters()
# => 102 million parameters

110105658

In [4]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

# Initializing Tokenizer

In [9]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("./thwiki-sentencepiecebpe.tokenizer.json")
encoded =  tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
print(encoded.ids)
print(encoded.tokens)

[15906, 1204, 18091, 13184, 1257, 12938, 3426, 9773, 5391, 1005, 4462, 1397, 1096, 1751, 1364, 527, 7777, 5202, 4057, 10411, 1564, 1317, 18091, 527, 8768, 3127, 1564, 6927, 2113, 1739]
['▁สวัส', 'ดี', 'ครับ', '▁ผม', 'ชื่อ', 'ไนท์', '▁ตอน', 'นี้ก็', 'เป็นเวลา', 'ที่', 'ผม', 'ต้อง', 'ไป', 'โรงเรียน', 'แล้ว', '▁', '▁นี่', 'คือการ', 'เว้น', 'วรรค', 'สอง', 'ที', 'ครับ', '▁', '▁จะได้', 'ออกเป็น', 'สอง', '▁Sp', 'ac', 'es']


In [33]:
tokenizer.enable_truncation(max_length=128)

In [51]:
encoded =  tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง SpacesWhat is great is that our tokenizer is optimized for Esperanto. Compared to a generic tokenizer trained for English, more native words are represented by a single, unsplit token. Diacritics, i.e. accented characters used in Esperanto – ĉ, ĝ, ĥ, ĵ, ŝ, and ŭ – are encoded natively. We also represent sequences in a more efficient manner. Here on this corpus, the average length of encoded sequences is ~30% smaller as when using the pretrained GPT-2 tokenizer.")
print("This will not be over 128: ", len(encoded.ids), encoded.tokens)
print(encoded.overflowing[0].tokens)

This will not be over 128:  128 ['▁สวัส', 'ดี', 'ครับ', '▁ผม', 'ชื่อ', 'ไนท์', '▁ตอน', 'นี้ก็', 'เป็นเวลา', 'ที่', 'ผม', 'ต้อง', 'ไป', 'โรงเรียน', 'แล้ว', '▁', '▁นี่', 'คือการ', 'เว้น', 'วรรค', 'สอง', 'ที', 'ครับ', '▁', '▁จะได้', 'ออกเป็น', 'สอง', '▁Sp', 'ac', 'es', 'W', 'h', 'at', '▁is', '▁g', 'reat', '▁is', '▁th', 'at', '▁o', 'ur', '▁to', 'k', 'en', 'iz', 'er', '▁is', '▁o', 'p', 'tim', 'iz', 'ed', '▁for', '▁E', 's', 'per', 'ant', 'o', '.', '▁Comp', 'ar', 'ed', '▁to', '▁a', '▁g', 'ener', 'ic', '▁to', 'k', 'en', 'iz', 'er', '▁tr', 'ain', 'ed', '▁for', '▁Eng', 'l', 'ish', ',', '▁m', 'ore', '▁n', 'ative', '▁w', 'ord', 's', '▁', 'are', '▁re', 'p', 'res', 'ent', 'ed', '▁by', '▁a', '▁s', 'ing', 'le', ',', '▁un', 's', 'pl', 'it', '▁to', 'k', 'en', '.', '▁D', 'ia', 'c', 'ri', 'tic', 's,', '▁i', '.', 'e', '.', '▁ac', 'c', 'ent', 'ed', '▁ch', 'ar', 'ac', 'ter', 's', '▁']
['us', 'ed', '▁in', '▁E', 's', 'per', 'ant', 'o', '▁–', '▁,', '▁,', '▁,', '▁,', '▁,', '▁and', '▁', '▁–', '▁', 'are', '▁', 'en

wrap tokenizers inside a PreTrainedTokenizerFast from transformers 

https://github.com/huggingface/tokenizers/issues/259

In [102]:
from tokenizers import SentencePieceBPETokenizer
from transformers import PreTrainedTokenizerFast


class SentencePieceBPETokenizerFast(PreTrainedTokenizerFast):
    def __init__(
        self,
        vocab_file,
        merges_file,
        bos_token="<s>",
        eos_token="</s>",
        sep_token="</s>",
        cls_token="<s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
        **kwargs
    ):
        super().__init__(
            SentencePieceBPETokenizer(
                vocab_file=vocab_file,
                merges_file=merges_file,
            ),
             bos_token=bos_token,
            eos_token=eos_token,
            unk_token=unk_token,
            sep_token=sep_token,
            cls_token=cls_token,
            pad_token=pad_token,
            mask_token=mask_token,
            **kwargs,
        )

In [5]:
import json

# with open("./thwiki-sentencepiecebpe.tokenizer.json", 'r' ) as json_data:
with open("./thwiki-charbpe-30522.tokenizer.json", 'r' ) as json_data:
     data = json.load(json_data)
vocab = data['model']['vocab']
merges = data['model']['merges']


with open('vocab.json', 'w', encoding='utf-8') as json_file:
    json.dump(vocab, json_file, ensure_ascii=False)
with open('merges.txt', 'w', encoding='utf-8') as f:
    for merge_string in merges:
        f.write(f'{merge_string}\n')

In [186]:
pretrain_tokenizer = SentencePieceBPETokenizerFast(vocab_file='vocab.json',merges_file ='merges.txt' )

In [187]:
dir(pretrain_tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_bos_token',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_encode_plus',
 '_eos_token',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_mask_token',
 '_maybe_update_backend',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_sep_token',
 '_tokenizer',
 '_unk_token',
 'add_special_tokens',
 'add_tokens',
 'additional_special_tokens',
 'additional_special_tokens_ids',
 'all_special_ids',
 'all_special_tokens',
 'backend_tok

In [141]:
# from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
# from tokenizers import Tokenizer
# from tokenizers.implementations import BaseTokenizer

# tokenizer = Tokenizer.from_file("./thwiki-sentencepiecebpe.tokenizer.json")
# base_tokenizer = BaseTokenizer(tokenizer) # Wrapper!! to PretrainTokenizerFast Tokenizer should be an instance of a Tokenizer provided by HuggingFace tokenizers library.
# # base_tokenizer = SentencePieceBPETokenizer()
# pretrain_tokenizer = PreTrainedTokenizerFast(tokenizer=base_tokenizer)
# pretrain_tokenizer

In [166]:
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
from tokenizers import Tokenizer, CharBPETokenizer
from tokenizers.implementations import BaseTokenizer

# tokenizer = Tokenizer.from_file("./thwiki-charbpe-30522.tokenizer.json")
# base_tokenizer = BaseTokenizer(tokenizer) # Wrapper!! to PretrainTokenizerFast Tokenizer should be an instance of a Tokenizer provided by HuggingFace tokenizers library.
base_tokenizer = CharBPETokenizer(vocab_file='vocab.json',merges_file ='merges.txt')
pretrain_tokenizer = PreTrainedTokenizerFast(tokenizer=base_tokenizer)
pretrain_tokenizer

In [158]:

from transformers import RobertaTokenizerFast
pretrain_tokenizer = RobertaTokenizerFast(vocab_file='vocab.json',merges_file ='merges.txt', max_len=512)

In [174]:
from transformers import RobertaTokenizerFast

pretrain_tokenizer = RobertaTokenizerFast.from_pretrained(".", max_len=512)

# Building our dataset

Build it with `from torch.utils.data.dataset import Dataset` just like [TextDataset](https://github.com/huggingface/transformers/blob/448c467256332e4be8c122a159b482c1ef039b98/src/transformers/data/datasets/language_modeling.py) and [LineByLineTextDataset](https://github.com/huggingface/transformers/blob/448c467256332e4be8c122a159b482c1ef039b98/src/transformers/data/datasets/language_modeling.py)

padding documentation [link](https://github.com/huggingface/tokenizers/blob/master/bindings/python/tokenizers/implementations/base_tokenizer.py#L52)

Potential Improvements
- การทำให้ Dataset นั้น dynamically tokenize + dynamically open file : ตอนนี้เวลาทำ Dataset จาก torch.utils.data.dataset จะทำการ tokenize เลยตอนอยู่ใน constructor  , กำลังคิดว่าถ้าเกิดว่า Data ใหญ่มากๆ อาจจะไม่เหมาะสมกับการทำแบบนี้  เพราะว่า Ram จะต้องมีขนาดเท่าๆกับ data ที่เราใส่เข้าไป  ซึ่งเป็นไปได้ยากหาก Data มีขนาดใหญ่มากๆ   ผมได้ทำการ Search ดูแล้วก็พบว่าจาก Discussion Forum ของ Pytorch: https://discuss.pytorch.org/t/how-to-use-a-huge-line-corpus-text-with-dataset-dataloader/30872 
Option1: ใช้ pd.Dataframe ในการเปิด File แบบ small chunks of data https://discuss.pytorch.org/t/data-processing-as-a-batch-way/14154/4?u=ptrblck
Option2: ใช้ byte Offsets จากไฟล์ใหญ่ๆเพื่อที่จะ lookup .seek(): https://github.com/pytorch/text/issues/130#issuecomment-510412877
More Examples: https://github.com/pytorch/text/blob/master/torchtext/datasets/unsupervised_learning.py , https://github.com/pytorch/text/blob/a5880a3da7928dd7dd529507eec943a307204de7/examples/text_classification/iterable_train.py#L169-L214

In [18]:


class THWikiDataset(Dataset):
    def __init__(self, evaluate = False, block_size=32):
#         tokenizer = Tokenizer.from_file("./thwiki-charbpe-30522.tokenizer.json")
#         tokenizer = CharBPETokenizer(vocab_file='vocab.json',merges_file ='merges.txt' )
        self.tokenizer = RobertaTokenizerFast(vocab_file='vocab.json',merges_file ='merges.txt', max_len=512)
#         no_accent_strip = BertNormalizer(strip_accents=False)
#         tokenizer._tokenizer.normalizer = no_accent_strip
#         tokenizer._tokenizer.post_processor = BertProcessing(
#             ("</s>", tokenizer.token_to_id("</s>")),
#             ("<s>", tokenizer.token_to_id("<s>")),
#         )
#         tokenizer.enable_truncation(max_length=512)
#         tokenizer.enable_padding()
        # or use the RobertaTokenizer from `transformers` directly.

        self.examples = []
        
        # Use AA and AB as train, AC as test
        src_files = Path("../data/text/AC/").glob("wiki*") if evaluate else Path("../data/text/").glob("A[AB]/wiki*")
        for src_file in src_files:
            print("🔥", src_file)
            lines = src_file.read_text(encoding="utf-8").splitlines()
#             self.examples += [x.ids for x in tokenizer.encode_batch(lines)]
            batch_encoding = tokenizer.batch_encode_plus(lines, add_special_tokens=True, max_length=block_size)
            self.examples = batch_encoding["input_ids"]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, i):
        # We’ll pad at the batch level.
        return torch.tensor(self.examples[i], dtype=torch.long)

In [19]:
dataset = THWikiDataset()

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


🔥 ../data/text/AA/wiki_04
🔥 ../data/text/AA/wiki_95
🔥 ../data/text/AA/wiki_12


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


🔥 ../data/text/AA/wiki_26
🔥 ../data/text/AA/wiki_61


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


...

🔥 ../data/text/AB/wiki_55
🔥 ../data/text/AB/wiki_88


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.
Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


🔥 ../data/text/AB/wiki_82
🔥 ../data/text/AB/wiki_29


Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


🔥 ../data/text/AB/wiki_50


In [274]:

dataloader = DataLoader(dataset, batch_size=1, collate_fn=data_collator,
                        shuffle=True, num_workers=4) # Still cant make more batch size!! Need collate function!

In [275]:
for i_batch, sample_batched in enumerate(dataloader):
    print(i_batch, sample_batched)
    oumodel()

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/opt/conda/lib/python3.7/site-packages/transformers/data/data_collator.py", line 79, in __call__
    inputs, labels = self.mask_tokens(batch)
  File "/opt/conda/lib/python3.7/site-packages/transformers/data/data_collator.py", line 102, in mask_tokens
    if self.tokenizer.mask_token is None:
AttributeError: 'CharBPETokenizer' object has no attribute 'mask_token'


In [276]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_parameters',
 '_tokenizer',
 'add_special_tokens',
 'add_tokens',
 'decode',
 'decode_batch',
 'enable_padding',
 'enable_truncation',
 'encode',
 'encode_batch',
 'get_vocab',
 'get_vocab_size',
 'id_to_token',
 'no_padding',
 'no_truncation',
 'normalize',
 'num_special_tokens_to_add',
 'padding',
 'post_process',
 'save',
 'save_model',
 'to_str',
 'token_to_id',
 'train',
 'truncation']

In [261]:
tokenizer = CharBPETokenizer(vocab_file='vocab.json',merges_file ='merges.txt' )
no_accent_strip = BertNormalizer(strip_accents=False)
tokenizer._tokenizer.normalizer = no_accent_strip
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)

input_ids = torch.tensor(tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces").ids).unsqueeze(0)
print(input_ids)
outputs = model(input_ids, labels=input_ids)
print(outputs)
loss, prediction_scores = outputs[:2]
print(loss, prediction_scores.shape)

tensor([[    0, 27045,  1038,   340,     3,  3819,  1223,  2492,   314,     3,
          1561,  9379,  4160,  1009,  3819, 18348,  1524,  1016,   359,     3,
          3691,  4045,  3580,  9717,  1404,  1275,  1038,   340,     3,  3038,
          2891, 12364,     3, 15536,     3,     2]])
(tensor(10.4711, grad_fn=<NllLossBackward>), tensor([[[-0.1541, -0.1424, -0.0832,  ...,  0.5985, -0.6984, -0.8883],
         [ 0.2593, -0.3135, -0.0564,  ...,  1.0348, -0.1579, -0.4282],
         [-0.4429, -0.2429,  0.3989,  ...,  0.6725, -0.0034, -0.2337],
         ...,
         [-0.0181, -0.6908, -0.4555,  ...,  1.0229, -1.0689, -0.6037],
         [-0.6513, -0.3325,  0.3148,  ..., -0.0385, -0.4886, -0.4097],
         [ 0.1321, -0.1464, -0.5289,  ...,  0.7893, -0.2908, -0.4264]]],
       grad_fn=<AddBackward0>))
tensor(10.4711, grad_fn=<NllLossBackward>) torch.Size([1, 36, 30522])


In [264]:
dataset.__getitem__(1).unsqueeze(0)

tensor([[   0, 1153, 3909, 1043,  333,    3,    2]])

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1

In [234]:
# %%time
# from transformers import TextDataset, LineByLineTextDataset

# # dataset = LineByLineTextDataset(
# #     tokenizer=pretrain_tokenizer,
# #     file_path="../data/text/AA/wiki_01",
# #     block_size=128,
# # )

# dataset = TextDataset(
#     tokenizer=pretrain_tokenizer,
#     file_path="../data/text/AA/wiki_01",
#     block_size=128,
# )


In [215]:
one_doc = list(Path("../data/text/AA/").glob("wiki*"))[0].read_text(encoding="utf-8").splitlines()
tokenizer = Tokenizer.from_file("./thwiki-sentencepiecebpe.tokenizer.json")
tokenizer.encode_batch(one_doc[:8])

[Encoding(num_tokens=8, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=1, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=1, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=103, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=1, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=154, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=1, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=189, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])]

In [17]:
one_doc = list(Path("../data/text/AA/").glob("wiki*"))[0].read_text(encoding="utf-8").splitlines()
tokenizer = RobertaTokenizerFast(vocab_file='vocab.json',merges_file ='merges.txt', max_len=512)
tokenizer.batch_encode_plus(one_doc[:8])

{'input_ids': [[0, 32, 1096, 1085, 33, 6, 5273, 6, 1211, 33, 6, 1207, 30, 19, 19, 1141, 18, 1210, 18, 1200, 19, 1095, 35, 1212, 33, 5273, 6, 1209, 33, 6, 86, 86, 72, 86, 76, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 6, 34, 2], [0, 86, 86, 72, 86, 76, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 2], [0, 2], [0, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 12, 31, 13, 86, 74, 86, 72, 86, 81, 86, 86, 86, 81, 86, 86, 86, 86, 78, 86, 86, 86, 78, 86, 72, 86, 73, 86, 86, 78, 86, 86, 78, 86, 86, 78, 86, 72, 86, 86, 72, 86, 77, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 86, 12, 31, 13, 86, 86, 86, 86, 86, 86, 72, 86, 76, 86, 86, 86, 86, 86, 86, 86, 72, 86, 78, 86, 86, 86, 80, 86, 86, 86, 86, 86, 86, 74, 86, 86, 86, 86, 86, 86, 78, 86, 76, 86, 86, 86, 126, 86, 86, 86, 86, 126, 86, 78, 86, 86, 86, 86, 86, 86, 80, 86, 86, 86, 76, 86, 73, 86, 77, 86, 86, 86, 86, 86, 86, 124, 86, 80, 86, 86, 86, 86, 86, 124, 86, 86, 77, 86, 124, 86, 86, 86, 86, 86, 86, 86, 86, 86, 74, 86, 78, 86, 86, 86, 86

In [221]:
print(tokenizer.encode_batch(one_doc[:8])[5].tokens)

['▁ฟิลิปปินส์', 'ตั้งอยู่ใน', 'แถบ', 'วงแหวน', 'แห่ง', 'ไฟ', 'และ', 'ใกล้กับ', 'เส้น', 'ศูนย์สูตร', '▁ทําให้มี', 'แนวโน้ม', 'สูง', 'ที่จะ', 'ประสบ', 'ภัย', 'จาก', 'แผ่นดินไหว', 'และ', 'ไต้ฝุ่น', '▁แต่ก็', 'ทําให้', 'มีทั้ง', 'ทรัพยากรธรรมชาติ', 'ที่', 'อุดมสมบูรณ์', 'และความ', 'หลากหลาย', 'ทางชีวภาพ', 'อย่างยิ่ง', 'เช่นกัน', '▁ฟิลิปปินส์', 'มี', 'เนื้อที่', 'ประมาณ', '▁300', ',000', '▁ตารางกิโลเมตร', '▁(1', '15', ',8', '31', '▁ตาราง', 'ไมล์', ')', '▁และมี', 'ประชากร', 'ประมาณ', '▁100', '▁ล้านคน', '▁นับเป็น', 'ประเทศ', 'ที่มีประชากร', 'มากที่สุด', 'เป็นอันดับที่', '▁8', '▁ใน', 'เอเชีย', '▁และเป็น', 'ประเทศ', 'ที่มีประชากร', 'มากที่สุด', 'เป็นอันดับที่', '▁12', '▁ของโลก', '▁นอกจากนี้', '▁ณ', '▁ปี', '▁พ.ศ.', '▁2556', '▁ยังมี', 'ชาว', 'ฟิลิปปินส์', 'อีก', 'ประมาณ', '▁10', '▁ล้านคน', 'อาศัยอยู่ใน', 'ต่างประเทศ', '▁รวม', 'แล้ว', 'ถือเป็น', 'กลุ่มคน', 'พลัด', 'ถิ่น', 'ที่ใหญ่ที่สุด', 'กลุ่ม', 'หนึ่งของ', 'โลก', '▁ความ', 'หลากหลาย', 'ทาง', 'ชาติพันธุ์', 'และวัฒนธรรม', 'ปรากฏ', 'ให้เห็น', 'ตลอด

In [211]:
one_doc[:8]

['<doc id="1990" url="https://th.wikipedia.org/wiki?curid=1990" title="ประเทศฟิลิปปินส์">',
 'ประเทศฟิลิปปินส์',
 '',
 'ฟิลิปปินส์ (; ) หรือชื่อทางการว่า สาธารณรัฐฟิลิปปินส์ (; ) เป็นประเทศเอกราชที่เป็นหมู่เกาะในภูมิภาคเอเชียตะวันออกเฉียงใต้ ตั้งอยู่ในมหาสมุทรแปซิฟิกตะวันตก ประกอบด้วยเกาะ 7,641 เกาะ ซึ่งจัดอยู่ในเขตภูมิศาสตร์ใหญ่ 3 เขตจากเหนือจรดใต้ ได้แก่ ลูซอน วิซายัส และมินดาเนา เมืองหลวงของประเทศคือมะนิลา ส่วนเมืองที่มีประชากรมากที่สุดคือนครเกซอน ทั้งสองเป็นส่วนหนึ่งของเมโทรมะนิลา ฟิลิปปินส์มีอาณาเขตติดต่อกับทะเลจีนใต้ทางทิศตะวันตก ทะเลฟิลิปปินทางทิศตะวันออก และทะเลเซเลบีสทางทิศตะวันตกเฉียงใต้ โดยมีพรมแดนทางทะเลร่วมกับไต้หวันทางทิศเหนือ ปาเลาทางทิศตะวันออก มาเลเซียและอินโดนีเซียทางทิศใต้ และเวียดนามทางทิศตะวันตก ',
 '',
 'ฟิลิปปินส์ตั้งอยู่ในแถบวงแหวนแห่งไฟและใกล้กับเส้นศูนย์สูตร ทำให้มีแนวโน้มสูงที่จะประสบภัยจากแผ่นดินไหวและไต้ฝุ่น แต่ก็ทำให้มีทั้งทรัพยากรธรรมชาติที่อุดมสมบูรณ์และความหลากหลายทางชีวภาพอย่างยิ่งเช่นกัน ฟิลิปปินส์มีเนื้อที่ประมาณ 300,000 ตารางกิโลเมตร (115,831 ตารางไ

In [25]:
from transformers import DataCollatorForLanguageModeling
tokenizer = RobertaTokenizer(vocab_file='vocab.json',merges_file ='merges.txt', max_len=512)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [26]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_bos_token',
 '_cls_token',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_encode_plus',
 '_eos_token',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_mask_token',
 '_maybe_update_backend',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_prepare_for_model',
 '_sep_token',
 '_tokenize',
 '_unk_token',
 'add_special_tokens',
 'add_tokens',
 'added_tokens_decoder',
 'added_tokens_encoder',
 'additional_spe

In [29]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./test_RoBERTa",
    overwrite_output_dir=True,
    num_train_epochs=20,
    per_device_train_batch_size=256,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
%%time
trainer.train()

In [31]:
trainer.save_model("./EsperBERTo")

In [180]:
encoded = pretrain_tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
encoded

[0,
 100,
 104,
 107,
 99,
 104,
 108,
 107,
 100,
 105,
 107,
 99,
 99,
 107,
 100,
 107,
 108,
 105,
 99,
 100,
 100,
 99,
 99,
 107,
 99,
 104,
 103,
 9042,
 2771,
 3174,
 2]